In [123]:
!pip install kaggle
!pip install pandas
!pip install graphviz
!pip3 install pydotplus
!pip install webcolors
!pip install colour
import os
import zipfile
import kaggle

In [27]:
#{"username":"anguis1","key":"8e5aa869bd2b736636371bd2e59902b3"}

os.environ['KAGGLE_USERNAME'] = 'anguis1'
os.environ['KAGGLE_KEY'] = '8e5aa869bd2b736636371bd2e59902b3'

#Nom du dataset
dataset = "vishalsubbiah/pokemon-images-and-types"

#Téléchargement du Dataset
kaggle.api.dataset_download_files(dataset)

with zipfile.ZipFile('pokemon-images-and-types.zip', 'r') as zip_ref:
    zip_ref.extractall('pokemon')

In [28]:
import urllib.request
import json
import pandas as pd

#On charge le fichier csv
df = pd.read_csv('./pokemon/pokemon.csv')

#On crée un dictionnaire associant le nom du pokemon à son image
pokemon_images = {}
for filename in os.listdir("./pokemon/images/images"):
    if filename.endswith(".png"):
        name = filename.split(".")[0]
        pokemon_images[name] = os.path.join("pokemon/images/images", filename)

# Ajouter une colonne "image" au dataframe
df["image"] = df["Name"].apply(lambda x: pokemon_images.get(x, None))

# Supprimer les lignes sans image
df = df.dropna(subset=["image"])

# Afficher le dataframe
print(df.head())

         Name  Type1   Type2                                 image
0   bulbasaur  Grass  Poison   pokemon/images/images\bulbasaur.png
1     ivysaur  Grass  Poison     pokemon/images/images\ivysaur.png
2    venusaur  Grass  Poison    pokemon/images/images\venusaur.png
3  charmander   Fire     NaN  pokemon/images/images\charmander.png
4  charmeleon   Fire     NaN  pokemon/images/images\charmeleon.png


In [102]:
import numpy as np
from sklearn.cluster import KMeans

def get_dominant_colors(image_path, n_colors):
    
    #Définir explicitement n_init pour éviter l'avertissement de version future
    n_init = 'auto'
    
    # Charger l'image et la convertir en une matrice numpy
    img = Image.open(image_path)
    img = np.array(img)

    # Obtenir un tableau de pixels en deux dimensions
    pixels = img.reshape(-1, 3)
    #-1 car l'argument s'adapte à la taille de l'image et 3 pour le RGB
    
    # Créer un objet de clustering
    kmeans = KMeans(n_clusters=n_colors, n_init=n_init)

    # Entraîner le modèle
    kmeans.fit(pixels)

    # Récupérer les couleurs dominantes
    dominant_colors = kmeans.cluster_centers_

    # Convertir les couleurs en entiers et les ajouter à une liste
    dominant_colors = [tuple(map(int, color)) for color in dominant_colors]
    
    return dominant_colors

In [103]:
def rgb_to_hex(rgb):
    #Convertir les valeurs RGB en valeurs hexadécimales
    hex_color = '#{:02x}{:02x}{:02x}'.format(int(rgb[0]), int(rgb[1]), int(rgb[2]))
    return hex_color

In [104]:
def rgb_to_hex_list(image_path, n_colors):
    
    #Fonction permettant de convertir une couleur rgb (liste de 3 valeurs) en une valeur hexadécimale
    
    dominant_colors = get_dominant_colors(image_path, n_colors)
    
    #Convertir la liste de couleurs RGB en liste de couleurs hexadécimales
    hex_list = [rgb_to_hex(rgb) for rgb in dominant_colors]
    return hex_list

In [127]:
import webcolors
from colour import Color

def hex_to_color_name(requested_color):
    # Convertir la couleur en un objet Color
    try:
        requested_color = Color(requested_color)
    except:
        raise ValueError(f"La valeur hexadécimale '{requested_color}' n'est pas valide.")
    
    # Trouver la couleur la plus proche dans la liste des noms de couleurs prédéfinis
    closest_color = None
    min_colors = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        try:
            # Convertir la couleur prédéfinie en un objet Color
            color = Color(key)
            
            # Calculer la distance entre les deux couleurs en utilisant la différence Euclidienne
            r_distance = (requested_color.red - color.red) ** 2
            g_distance = (requested_color.green - color.green) ** 2
            b_distance = (requested_color.blue - color.blue) ** 2
            distance = r_distance + g_distance + b_distance
            
            # Garder la distance minimale pour chaque couleur prédéfinie
            if distance not in min_colors:
                min_colors[distance] = name
        except ValueError:
            # Ignorer les couleurs prédéfinies qui ne peuvent pas être converties en objet Color
            pass
    
    # Retourner la couleur prédéfinie ayant la distance minimale à la couleur demandée
    closest_color = min_colors[min(min_colors.keys())]
    return closest_color


In [128]:
# import random
import random

def satisfied_or_not():
    
    #Fonction pour générer un avis aléatoire (positif ou négatif) de l'utilisateur pour pouvoir effectuer des tests
    
    return random.choice(['Yes', 'No'])

In [130]:
import os
import csv
import json
from PIL import Image

# Chemin du dossier contenant les images
folder_path = './pokemon/images/images'

# Liste des extensions de fichier que nous voulons inclure dans l'analyse
extensions = ['.jpg', '.jpeg', '.png']

# Initialisation de la liste pour stocker les informations de chaque image
list_images_info = []

# Ouverture du fichier CSV
with open('./pokemon/pokemon.csv') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader) # Ignorer la première ligne (en-tête)
    for row in csv_reader:
        # Récupère les informations du CSV
        pokemon_name = row[0]
        type1 = row[1]
        try :
            type2 = row[2]
        except Exception as e:
            type2 = 'None'
            continue

        filename = row[0]+'.png'
    
        try:
            # Vérifie si le fichier est une image avec une extension spécifiée
            if any(filename.lower().endswith(ext) for ext in extensions):
                # Chemin complet de l'image
                filepath = os.path.join(folder_path, filename)
                # Ouvre l'image avec PIL
                with Image.open(filepath) as img:
                    
                    #On calcule les kmeans de l'image
                    kmeans_hex_list = rgb_to_hex_list(filepath, 5)
                    
                    # Stocke les informations dans un dictionnaire
                    images_info = {'filename': filename,
                                   'pokemon_name': pokemon_name,
                                   'type1': type1,
                                   'type2': type2,
                                   'size': str(os.path.getsize(filepath)),
                                   'k-means_region_one' : hex_to_color_name(kmeans_hex_list[1]),
                                   'k-means_region_two' : hex_to_color_name(kmeans_hex_list[2]),
                                   'k-means_region_three' : hex_to_color_name(kmeans_hex_list[3]),
                                   'k-means_region_four' : hex_to_color_name(kmeans_hex_list[4]),
                                   'favorite_utilisateur1': satisfied_or_not(),
                                   'favorite_utilisateur2': satisfied_or_not(),
                                   'favorite_utilisateur3': satisfied_or_not(),
                                   'favorite_utilisateur4': satisfied_or_not()}
                    # Ajoute le dictionnaire à la liste
                    list_images_info.append(images_info)
        except Exception as e:
            filename = row[0]+'.jpg' #Si le fichier n'existe pas en .png, on teste de le trouver en .jpg
    
            try:
                # Vérifie si le fichier est une image avec une extension spécifiée
                if any(filename.lower().endswith(ext) for ext in extensions):
                    # Chemin complet de l'image
                    filepath = os.path.join(folder_path, filename)
                    # Ouvre l'image avec PIL
                    with Image.open(filepath) as img:

                        #On calcule les kmeans de l'image
                        kmeans_hex_list = rgb_to_hex_list(filepath, 5)

                        # Stocke les informations dans un dictionnaire
                        images_info = {'filename': filename,
                                       'pokemon_name': pokemon_name,
                                       'type1': type1,
                                       'type2': type2,
                                       'size': str(os.path.getsize(filepath)),
                                       'k-means_region_one' : hex_to_color_name(kmeans_hex_list[1]),
                                       'k-means_region_two' : hex_to_color_name(kmeans_hex_list[2]),
                                       'k-means_region_three' : hex_to_color_name(kmeans_hex_list[3]),
                                       'k-means_region_four' : hex_to_color_name(kmeans_hex_list[4]),
                                       'favorite_utilisateur1': satisfied_or_not(),
                                       'favorite_utilisateur2': satisfied_or_not(),
                                       'favorite_utilisateur3': satisfied_or_not(),
                                       'favorite_utilisateur4': satisfied_or_not()}
                        # Ajoute le dictionnaire à la liste
                        list_images_info.append(images_info)
            except Exception as e:
                print(f"Erreur lors de l'extraction des informations de {row[0]}: {e}")
                continue

# Écriture du résultat dans un fichier JSON
with open('images_info.json', 'w') as json_file:
    json.dump(list_images_info, json_file)

In [83]:
from sklearn import tree
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import graphviz
import pydotplus
from IPython.display import Image, display

with open('./images_info.json') as d:
    file_contents = d.read()
data = json.loads(file_contents)
datadict={}
resultdict={}

favorite_all=[
        "favorite_utilisateur2",
        "favorite_utilisateur3",
        "favorite_utilisateur4",
             "k-means_regions"]

favorite=["favorite_utilisateur1"]
for keyi in data[0].keys(): 
    if keyi not in favorite_all:
        if keyi in favorite:
            resultdict[keyi]=[]
        else:
            datadict[keyi]=[]

for dicti in data:
    for keyi in dicti.keys():
        if keyi not in favorite_all:
            if keyi in favorite:
                resultdict[keyi].append(dicti[keyi])
            else:
                datadict[keyi].append(dicti[keyi])


#creating dataframes
dataframe = pd.DataFrame(datadict)
resultframe = pd.DataFrame(resultdict)

print(dataframe)
print(resultframe)

#generating numerical labels
le1 = LabelEncoder() #On associe un chiffre avec le texte du tag
dataframe['filename'] = le1.fit_transform(dataframe['filename'])

le2 = LabelEncoder()
dataframe['pokemon_name'] = le2.fit_transform(dataframe['pokemon_name'])

le3 = LabelEncoder()
dataframe['type1'] = le3.fit_transform(dataframe['type1'])

le4 = LabelEncoder()
dataframe['type2'] = le4.fit_transform(dataframe['type2'])

le5 = LabelEncoder() #On associe un chiffre avec le texte du tag
dataframe['size'] = le5.fit_transform(dataframe['size'])

le6 = LabelEncoder()
dataframe['k-means_region_one'] = le6.fit_transform(dataframe['k-means_region_one'])

le7 = LabelEncoder()
dataframe['k-means_region_two'] = le7.fit_transform(dataframe['k-means_region_two'])

le8 = LabelEncoder()
resultframe['favorite_utilisateur1'] = le8.fit_transform(resultframe['favorite_utilisateur1'])
"""
le9 = LabelEncoder()
resultframe['favorite_utilisateur2'] = le9.fit_transform(resultframe['favorite_utilisateur3'])

le10 = LabelEncoder()
resultframe['favorite_utilisateur3'] = le10.fit_transform(resultframe['favorite_utilisateur3'])

le11 = LabelEncoder()
resultframe['favorite_utilisateur4'] = le11.fit_transform(resultframe['favorite_utilisateur4'])
"""
#Use of decision tree classifiers
dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(dataframe, resultframe)

dot_data = tree.export_graphviz(dtc, out_file=None,
      feature_names=dataframe.columns,
      filled=True, rounded=True, 
      class_names =
      le5.inverse_transform(
        resultframe.favorite_utilisateur1.unique())
     ) 
graph = graphviz.Source(dot_data) 

pydot_graph = pydotplus.graph_from_dot_data(dot_data)
img = Image(pydot_graph.create_png())
display(img)

            filename pokemon_name     type1   type2  size k-means_region_one  \
0      bulbasaur.png    bulbasaur     Grass  Poison  2198             orange   
1        ivysaur.png      ivysaur     Grass  Poison  3188                red   
2       venusaur.png     venusaur     Grass  Poison  5770                red   
3      charizard.png    charizard      Fire  Flying  4980              black   
4     butterfree.png   butterfree       Bug  Flying  3982              olive   
..               ...          ...       ...     ...   ...                ...   
400     magearna.jpg     magearna     Steel   Fairy  1952            #a29b91   
401    marshadow.jpg    marshadow  Fighting   Ghost  1277            #fefefe   
402    naganadel.jpg    naganadel    Poison  Dragon  2798            #be9ccf   
403    stakataka.jpg    stakataka      Rock   Steel  2696            #fefefd   
404  blacephalon.jpg  blacephalon      Fire   Ghost  1971            #bebcbf   

    k-means_region_two  
0             

InvocationException: GraphViz's executables not found